In [ ]:
# Mount google drive to notebook.
from google.colab import drive
drive.mount('/content/drive/')

# To force remounting google drive to notebook.
#drive.mount('/content/drive', force_remount = True)

# Dismount google drive from notebook.
#drive.flush_and_unmount()

# Ensure we are in the correct working directory.
%cd /content/drive/MyDrive/CX4041-Group-Project/
%ls\

Mounted at /content/drive/
/content/drive/MyDrive/CX4041-Group-Project
 Augmented_Test_Images/   'CX4041 Group Project Report.docx'
 Augmented_Train_Images/   Model-Figures/
 Checkpoint/               Notebooks/
 CSV/                     '[Video Presentation] StoryBoard & Script.gdoc'


In [ ]:
# Import the required libraries.
import numpy as np
import pandas as pd
from glob import glob
import os 
import cv2
import random
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline

# Specify the filepaths for the train and test images.
train_dir = "/content/drive/MyDrive/CX4041-Group-Project/Augmented_Train_Images/"
test_dir = "/content/drive/MyDrive/CX4041-Group-Project/Renamed_Test_Images/"

In [ ]:
# Define the width and height of the resized images.
width = 299
height = 299

# Function to read in the training images for visualization.
def no_mask():
  images = []
  labels = []
  imgLabelArray = [[]]
  for f in os.listdir(train_dir):
    labels.append(f)
    for img in os.listdir(os.path.join(train_dir, f)):
      path = os.path.join(train_dir, f, img)
      img = cv2.imread(path)
      img = cv2.resize(img, (width, height))
      # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      # img = cv2.resize(img, (45, 45))
      # img = img.flatten()

      images.append(img)
      imgLabelArray.append([img, f])

  images = np.asarray(images)
  labels = np.asarray(labels)

  return images, labels, imgLabelArray

# Obtain the numpy arrays of the images and their labels.
images, labels, imgLabelArray = no_mask()

# for i in range(12):
#   plt.subplot(3,4, i+1)
#   plt.imshow(images[i])

In [ ]:
# Function rto perform masking of images.
def mask_plant(img):
  # Use gaussian blur
    blurImg = cv2.GaussianBlur(img, (5, 5), 0)   
    
    # Convert to HSV image
    hsvImg = cv2.cvtColor(blurImg, cv2.COLOR_BGR2HSV)  
    
    # Create mask (parameters - green color range)
    lower_green = (25, 40, 50)
    upper_green = (75, 255, 255)
    mask = cv2.inRange(hsvImg, lower_green, upper_green)  
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # Create bool mask
    bMask = mask > 0  
    
    # Apply the mask
    processedImg = np.zeros_like(img, np.uint8)  # Create empty image
    processedImg[bMask] = img[bMask]  # Apply boolean mask to the origin image

    # Plot a comparison of each image at each stage of preprocessing.
    # plt.subplot(2, 3, 1); plt.imshow(img)  # Show the original image
    # plt.subplot(2, 3, 2); plt.imshow(blurImg)  # Blur image
    # plt.subplot(2, 3, 3); plt.imshow(hsvImg)  # HSV image
    # plt.subplot(2, 3, 4); plt.imshow(mask)  # Mask
    # plt.subplot(2, 3, 5); plt.imshow(bMask)  # Boolean mask
    # plt.subplot(2, 3, 6); plt.imshow(processedImg)  # Image without background
    
    return processedImg

In [ ]:
# Load and mask the training images.
#imgLabelArray[0] is empty because the test images naming starts with an index of 1.
imgIndex = 0 
for label in labels:
  imgDir = '/content/drive/MyDrive/CX4041-Group-Project/Preprocessed_Train_Images/' + label + '/'
  if not os.path.isdir(imgDir):
      os.makedirs(imgDir)
  imgIndex = 0
  for img in imgLabelArray:
    if(img != [] and label == img[1]):
      img[0] = np.asarray(img[0])
      processedImg = mask_plant(img[0])
      processedImg = Image.fromarray(processedImg)
      imgName = label + '_' + str(imgIndex) + '.png'
      imgIndex = imgIndex + 1
      processedImg.save(imgDir + imgName)


In [ ]:
# Load the test images.
testPaths = glob(test_dir+'/test/*')
imageNameArray = []
testImages = []
for path in testPaths:
    imageNameArray.append(path.split('/')[-1])
    testImages.append(cv2.resize(cv2.imread(path),(width, height)))
testImages = np.asarray(testImages)
print('Number of test images read in:', len(imageNameArray))

Number of test images read in: 794


In [ ]:
# Mask the test Images.
testArray = []
maskedTestDir = '/content/drive/MyDrive/CX4041-Group-Project/Preprocessed_Test_Images/test/'
if not os.path.isdir(maskedTestDir):
    os.makedirs(maskedTestDir)
for i in range(0, len(imageNameArray)):
  testImages[i] = np.asarray(testImages[i])
  processedTestImg = mask_plant(testImages[i])
  testArray.append(processedTestImg)
  processedTestImg = Image.fromarray(processedTestImg)
  imgName = imageNameArray[i]
  processedTestImg.save(maskedTestDir + imgName)
print('Number of test images processed:', len(testArray))

Number of test images processed: 794
